In [2]:
import pandas as pd
from Entity.profile import Profile
from Configs.dbConfigs import FETCH_COMPANY_PEOPLE, FETCH_COMPANY_PEOPLE_FOR_OFFSET, INSERT_QUERY_PROFILE, PROFILE_TABLE_COLUMNS
from Configs.envrinomentSpecificConfgis import TABLE_NAME
from Configs.jobConfigs import COMPANY_NAME, KEYWORD_TITLE, MAX_LINKEDIN_API_CALLS_LIMIT, SEARCH_RESULT_LIMIT
from Main.login import Login
from Utils.dbUtils import insert_bulk_data, execute_get_command, readSQLQueryinPD
from Utils.utils import get_concatinated_df, get_df_items_for_search_results, get_overall_category, get_scheduler_config, get_uncommon_rows, remove_duplicates_from_df, update_schedulers_config


Your BASE_DIR is SET to: D:\study-code-repeat\coding\Linkedin-Scalable-invitations 


In [3]:
SEARCH_RESULT_LIMIT

5

In [ ]:
# LOGIN TO LINKEDIN
api = Login()

# Assuming your DataFrame is named df
category_searching = get_overall_category(KEYWORD_TITLE)


In [ ]:
category_searching

In [ ]:

# FETCH EXISTING DATA FROM GOOGLE SPREADSHEET
OFFSET_LIMIT = execute_get_command(FETCH_COMPANY_PEOPLE_FOR_OFFSET.format(TABLE_NAME, COMPANY_NAME.lower(), category_searching.lower()))[0][0]


In [ ]:
OFFSET_LIMIT

In [ ]:


# WE ARE ALLOWED TO MAKE AROUND 500 CALLS TO LINKEDIN PER HOURS TO PREVENT IT FROM MARKING AT AS A BOT
# WE WILL USE 350 AS PER HOUR LIMIT FOR SAFE SIDE
# TOTAL CALLS TO LIKNEDIN: LIMIT + 2 SO OUR LIMIT SHOULD NOT EXCEED 348 IN TOTAL
total_no_of_api_calls_to_lk = get_scheduler_config('total_number_of_api_calls_to_linkedin')

if (MAX_LINKEDIN_API_CALLS_LIMIT - total_no_of_api_calls_to_lk - SEARCH_RESULT_LIMIT) > 0:

    try:
        # CREATING AN EMPTY LIST TO STORE PEOPLE FROM SEARCH
        people_from_search_json_list = []

        # CALL TO SEARCH PEOPLE
        results = api.search_peoplev2(
                            keywords="Recruiters",
                            limit=SEARCH_RESULT_LIMIT,
                            offset=OFFSET_LIMIT,
                            currentCompany=[1441],
                            network=["S", "O"],
                            geoUrn=[103644278],
                            title=[],
                            resultType=["PEOPLE"],
                            # Keywords inside filters
                            company=[],
                            firstName=[],
                            lastName=None,
                            connectionOf=[],
                            followerOf=[],
                            # list of strings
                            talksAbout=[],
                            industry=[],
                            pastCompany=[],
                            profileLanguage=[],
                            schoolFilter=[],
                            schoolFreetext=[],
                            searchId=["b418ee28-c4de-4324-a085-ffe3525acddd"],
                            )

        # LOOPING OVER TO FETCH PEOPLE DATA FROM GET_PROFILE API AND PARSING RESULTS INTO OBJECT
        for peopleJSON in results:
            profile = Profile()
            
            # GET PROFILE
            peopleProfile = api.get_profile(peopleJSON["urn_id"])
            
            # PROFILE PARSING
            profile.parseProfileJSON(peopleProfile)
            profile.parseSearchPeopleJSON(peopleJSON)
            
            # APPENDING TO LIST
            people_from_search_json_list.append(get_df_items_for_search_results(profile))

        # CONVERTING LIST TO A DATAFRAME
        df_people_from_search = pd.DataFrame(people_from_search_json_list, columns=PROFILE_TABLE_COLUMNS)
        df_people_from_db = readSQLQueryinPD(FETCH_COMPANY_PEOPLE.format(TABLE_NAME, COMPANY_NAME.lower(), category_searching))
        
        # # GET THE NEW ROWS TO BE ADDED TO GOOGLE SPREADSHEET
        new_rows_added_df = get_uncommon_rows(df_people_from_db, df_people_from_search)
        # # REMOVE DUPLICATES
        no_duplicates_df = remove_duplicates_from_df(new_rows_added_df, unq_identifier_col='profile_urn_id')

        # # UPDATE DATA IN GOOGLE SPREADSHEET
        # gsheet.update_spreadsheet(no_duplicates_df)

        update_schedulers_config('total_number_of_api_calls_to_linkedin',  total_no_of_api_calls_to_lk + SEARCH_RESULT_LIMIT + 2)
    
    except e:
        print(f"Exception Occured:\n {e}")
        update_schedulers_config('total_number_of_api_calls_to_linkedin',  total_no_of_api_calls_to_lk)
else:
    print("Max api calls to linkedin exceeded. Try after an hour.")

In [ ]:
 new_rows_added_df = get_uncommon_rows(df_people_from_db, df_people_from_search)

In [ ]:
no_duplicates_df

In [ ]:
df_people_from_db

In [ ]:
insert_bulk_data(no_duplicates_df, INSERT_QUERY_PROFILE.format(TABLE_NAME))